In [7]:
from sklearn.metrics import classification_report
from os import walk, makedirs
import pandas as pd

# predictions_dir = '../predictions/'
# k_folds_dirs = next(walk(predictions_dir))[1]
# for fold in k_folds_dirs:
#   model_dirs = next(walk(f"{predictions_dir}/{fold}"))[1]
#   print(model_dirs)

['bert-base-multilingual-cased-sentiment-multilingual', 'xlm-roberta-base-sentiment-multilingual', 'dehatebert-mono-french', 'TFidf', 'french_xlm_xnli', 'Bert_embeddings']
['bert-base-multilingual-cased-sentiment-multilingual', 'xlm-roberta-base-sentiment-multilingual', 'dehatebert-mono-french', 'TFidf', 'french_xlm_xnli', 'Bert_embeddings']
['bert-base-multilingual-cased-sentiment-multilingual', 'xlm-roberta-base-sentiment-multilingual', 'dehatebert-mono-french', 'TFidf', 'french_xlm_xnli', 'Bert_embeddings']


In [ ]:
# def read_predictions_for_fold(k: int) -> dict:
#   data_dict = {}
#   k_fold_dir = f"../predictions/k_is_{str(k)}/"
#   # print (os.path(data_ready_dir))
#   filenames = []
#   for (_, _, name) in walk(k_fold_dir): 
#     filenames.extend(name)

#   for fname in filenames:
#     dataframe = pd.read_csv(k_fold_dir+fname).sample(frac=1)
#     dict_key = fname.split(".csv")[0]
#     data_dict.update({dict_key : dataframe})

#   return data_dict